In [6]:
import pandas as pd
import numpy as np 
import sqlite3
import re
from matplotlib import pyplot as plt

In [7]:
data = pd.read_csv('./train/OurDataset_train.csv')

In [8]:
data

,label,text,is_valid
0,shunil_gongopaddhay,থেকে কত দূরে চলে এসেছে ভরত। সে হেসে উঠল আপন মন...,False
1,humayun_ahmed,এতে ভয় কমে যায়। বল একটা গল্প।’ ‘তুমি বল।’ আনিস...,False
2,shomresh,হবে। ওই দেখুন ওর এক চোখ কানা। ডান দিকটা দিয়ে দ...,False
3,humayun_ahmed,"বললাম, আপনি ওর গায়ে হাত দিলেন কেন? ষণ্ডাগণ্ডা ...",False
4,humayun_ahmed,হত! আবার চাদর মুড়ি দিয়ে নিজেকে গুটিয়ে ফেলি। যে...,False
...,...,...,...
14369,robindronath,"একবার কমলের মূর্ছা ভাঙিল, মূর্ছা ভাঙিয়া মাতার ...",False
14370,humayun_ahmed,চলে যেতে পারে। যাওয়াটা কি ঠিক হবে? আপা নিশ্চয়ই...,False
14371,MZI,"পেটব্যথা করছে সার, বাসায় যেতে হবে এক্ষুণি। অঙ্...",False
14372,shomresh,পৌঁছে সে ডরোথিকে দেখতে পেল। এই সকালবেলায় নরম র...,False


In [10]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()

In [12]:
stopwords = pd.read_csv('./stopword/Stopwords.csv')

In [13]:
def to_encode(bangla_word):
    return bangla_word.encode('utf-8')


def to_decode(bangla_word):
    return bangla_word.decode('utf-8')

In [15]:
texts = data['text'].values

In [16]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [17]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if(to_encode(cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [20]:
X_train = pre_process(data['text'].values)
y_train = data['label'].values

In [19]:
labels=np.unique(y_train)

In [21]:
 from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB(alpha=.01)),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_train)

print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy normalized %s' % accuracy_score(y_pred, y_train, normalize=False))
print(classification_report(y_train, y_pred,target_names=labels))

accuracy 0.9883818004730763
accuracy normalized 14207
                     precision    recall  f1-score   support

                MZI       1.00      0.92      0.96       880
            bongkim       1.00      0.99      1.00       450
      humayun_ahmed       0.97      1.00      0.98      3612
manik_bandhopaddhay       1.00      0.97      0.99       376
             nazrul       1.00      0.96      0.98       179
 nihar_ronjon_gupta       1.00      0.98      0.99       381
       robindronath       0.99      0.99      0.99      1007
         shirshendu       0.99      0.99      0.99       838
           shomresh       1.00      1.00      1.00      1126
          shordindu       0.99      0.99      0.99       711
      shorotchandra       0.99      1.00      0.99      1051
      shottojit_roy       1.00      1.00      1.00       680
shunil_gongopaddhay       0.99      0.99      0.99      1570
        tarashonkor       0.99      1.00      0.99       620
     toslima_nasrin       1.00

In [22]:
data1 = pd.read_csv('./test/OurDataset_test.csv')

In [28]:
A_train = pre_process(data1['text'].values)

In [29]:
print(data1.keys())

Index(['label', 'text', 'is_valid'], dtype='object')


In [30]:
data1.label

0              zahir_rayhan
1       shunil_gongopaddhay
2             shorotchandra
3             humayun_ahmed
4       shunil_gongopaddhay
               ...         
3587              shordindu
3588    shunil_gongopaddhay
3589    shunil_gongopaddhay
3590     nihar_ronjon_gupta
3591          humayun_ahmed
Name: label, Length: 3592, dtype: object

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB(alpha=.01)),
              ])
X_train = pre_process(data['text'].values)
y_train = data['label'].values
labels=np.unique(y_train)
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(A_train)

print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy normalized %s' % accuracy_score(y_pred, y_train, normalize=False))
print(classification_report(y_train, y_pred,target_names=labels))

ValueError: Found input variables with inconsistent numbers of samples: [3592, 14374]